In [30]:
"""
Code for accommodating the graph-making need of ECON 3229-2 Money, Banking, and Financial Systems 

University of Missouri-Columbia

Fangda Wang

Fall 2017


https://github.com/vitanova
"""

# This file is aimed to demonstate the usefulness of the code I wrote before
# I changed several parameters in 1 min or 2, and a new graph is plotted as required

import pandas as pd
import numpy as np
import datetime
import matplotlib.pyplot as plt

#this is a third-party api for manipulating fred data
from fred import Fred

#first, use the api key to get access to fred, all data in dictionary format
fr = Fred(api_key='4bba000775edea91bc01729d217cf3da', response_type='dict') ## <--- replacing with your own key

#now, use the series id to get observations
cpi=fr.series.observations(series_id="GS10") ## <--- make change: series_id
aaa=fr.series.observations(series_id="TB3MS") ## <--- make change: ibid
nber=fr.series.observations(series_id="USREC") 

#convert the dictionary data to data frame
CPI=pd.DataFrame.from_dict(cpi)
AAA=pd.DataFrame.from_dict(aaa)
NBER=pd.DataFrame.from_dict(nber)

#rename columns
CPI.columns = 'date', 'end', 'start', 'gs10' ## <--- make change: series_name
AAA.columns = 'date', 'end', 'start', 'tb3ms' ## <--- make change: ibid
NBER.columns = 'date', 'end', 'start', 'nber'

#merge the two tables and use date as key
al=pd.merge(CPI, AAA, how='left', on='date')
al=pd.merge(al, NBER, how='left', on='date')
al=al.drop('end_x', axis=1)
al=al.drop('start_x', axis=1)
al=al.drop('end_y', axis=1)
al=al.drop('start_y', axis=1)
al=al.drop('end', axis=1)
al=al.drop('start', axis=1)

al.head()

C:\Anaconda3\lib\site-packages\fred\helpers\__init__.py:36: FutureWarning: convert_objects is deprecated.  Use the data-type specific converters pd.to_datetime, pd.to_timedelta and pd.to_numeric.
  frame = frame.convert_objects(convert_numeric=True)
C:\Anaconda3\lib\site-packages\pandas\core\internals.py:3224: FutureWarning: Passing in 'datetime64' dtype with no frequency is deprecated and will raise in a future version. Please pass in 'datetime64[ns]' instead.
  return self.apply('astype', dtype=dtype, **kwargs)


,date,gs10,tb3ms,nber
0,1953-04-01,2.83,2.19,0
1,1953-05-01,3.05,2.16,0
2,1953-06-01,3.11,2.11,0
3,1953-07-01,2.93,2.04,0
4,1953-08-01,2.95,2.04,1


In [35]:
#to reflect the up-to-date nature of data, we set end date as last available month in fred
#actually, we need data from 1977-07-01, but inflation rate need to be calculated based on last year
#so we get cpi data from 1976
start_date=datetime.date(1977, 8, 1) ## <--- make change: series_start_date
end_date=datetime.date(2017, 8, 1) ## <--- make change: series_end_date

#since we only plot quarterly data, these monthly observations should be transformed in frequency
index = pd.date_range(start=start_date, end=end_date, freq='3MS')
columns = ['A']
def diff_quarter(d1, d2):
    return (d2.year - d1.year) * 4 + int((d2.month - d1.month)/3)
n_rows=diff_quarter(start_date, end_date) + 1
data = np.array([np.arange(n_rows)]*1).T
df = pd.DataFrame(data, index=index, columns=columns)
df['date']=df.index

#now, use the quarterly format table to extract information from the fred data
new_one=pd.merge(df, al, how='left', on='date')
new_one=new_one.drop('A', axis=1)
new_one.head()

,date,gs10,tb3ms,nber
0,1977-08-01,7.40,5.49,0
1,1977-11-01,7.58,6.10,0
2,1978-02-01,8.03,6.45,0
3,1978-05-01,8.35,6.41,0
4,1978-08-01,8.41,7.08,0


In [36]:
def spread(x, y):
    return (x-y)
new_one['spread']=new_one.apply(lambda row: spread(row['gs10'], row['tb3ms']), axis=1)
new_one.set_index('date') ## <--- make change: maybe use other functions instead

#transform date format from yyyy-mm-dd to yyyy Qq
def tran_date(xx):
    return str(xx.year) + " Q" + str(1+int(xx.month/3))
new_one['new_date']=new_one.apply(lambda row: tran_date(row['date']), axis=1)
## <--- make change: maybe drop other rows instead
t=new_one
#add the zero line, prep two nber series to facilitate making the shaded area between them
t['zero']=0
t['+nber']=20*t['nber'] ## <--- make change: the value of multiplier should be greater than maximum of other series
t['-nber']=-t['+nber']
t.head()

,date,gs10,tb3ms,nber,spread,new_date,zero,+nber,-nber
0,1977-08-01,7.40,5.49,0,1.91,1977 Q3,0,0,0
1,1977-11-01,7.58,6.10,0,1.48,1977 Q4,0,0,0
2,1978-02-01,8.03,6.45,0,1.58,1978 Q1,0,0,0
3,1978-05-01,8.35,6.41,0,1.94,1978 Q2,0,0,0
4,1978-08-01,8.41,7.08,0,1.33,1978 Q3,0,0,0


In [39]:
#since fill_between mathod doesnt work well with datetype64, we need to transform it
finally_=t.date.values
fig, ax=plt.subplots(figsize=(16, 10))
ax.plot(finally_, t['gs10'], label='10-year bond', linewidth=3) ## <--- make change: series_name, series_label
ax.plot(finally_, t['tb3ms'], label='90-day bill', ls='dashed', linewidth=3) ## <--- make change
ax.plot(finally_, t['spread'], label='spread between gs10 and tb3ms', linewidth=3) ## <--- make change
ax.plot(finally_, t['zero'], color='black', label='_nolegend_')

#transform the shaded area into rectangle
import matplotlib.transforms as mtransforms
trans = mtransforms.blended_transform_factory(ax.transData, ax.transAxes)
ax.fill_between(finally_, t['+nber'], t['-nber'], facecolor='gray', interpolate=False, transform=trans, alpha=0.2)

#set the horizontal label as transformed data type
ax.set_xlim([t.date[t.index[0]], t.date[t.index[-1]]])
real_date=[]
trans_date=[]
for i in range(11): ## <--- make change: date displayed in the horizontal axis may vary
    real_date.append(t.date[t.index[16*i]])
    trans_date.append(t.new_date[t.index[16*i]])
ax.set_xticks(real_date)
ax.set_xticklabels(trans_date, fontsize=14)

ax.set_yticklabels(np.linspace(-2, 16, 10), fontsize=14) ## <--- make change: the range of y may varies
## <--- make change: sometimes it's better to use two vertical axis for two variables whose values are in different 
##        order of magnitudes
ax.grid()
ax.set_ylim(-2, 16) ## <--- make change: ibid
ax.set_ylabel("%", fontsize=16)
ax.legend(loc='best', fontsize=16)
ax.set_title("Yield Spread on U.S. Treasury Securities:  10-Year Bonds minus 90-day T Bills", fontsize=20, position=[.5, 1.03])
## <--- make change: graph title
plt.savefig('spread.jpg', dpi=300)